### displaying tables

In [0]:
%sql
--bronze table
--view of taxi_raw_records table
select * from taxi_raw_records limit 10;

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-13T21:47:53Z,2016-02-13T21:57:15Z,1.4,8.0,10103,10110
2016-02-13T18:29:09Z,2016-02-13T18:37:23Z,1.31,7.5,10023,10023
2016-02-06T19:40:58Z,2016-02-06T19:52:32Z,1.8,9.5,10001,10018
2016-02-12T19:06:43Z,2016-02-12T19:20:54Z,2.3,11.5,10044,10111
2016-02-23T10:27:56Z,2016-02-23T10:58:33Z,2.6,18.5,10199,10022
2016-02-13T00:41:43Z,2016-02-13T00:46:52Z,1.4,6.5,10023,10069
2016-02-18T23:49:53Z,2016-02-19T00:12:53Z,10.4,31.0,11371,10003
2016-02-18T20:21:45Z,2016-02-18T20:38:23Z,10.15,28.5,11371,11201
2016-02-03T10:47:50Z,2016-02-03T11:07:06Z,3.27,15.0,10014,10023
2016-02-19T01:26:39Z,2016-02-19T01:40:01Z,4.42,15.0,10003,11222


In [0]:
%sql
--silver table
--view of flagged_rides table 
select * from flagged_rides limit 10;


week,zip,fare_amount,trip_distance
2016-01-25T00:00:00Z,11109,52.0,3.0
2016-02-15T00:00:00Z,7311,60.0,2.0
2016-01-25T00:00:00Z,10065,52.0,1.12
2016-02-08T00:00:00Z,11422,52.0,0.2
2016-01-11T00:00:00Z,11422,52.0,8.7
2015-12-28T00:00:00Z,10023,52.0,0.3
2016-01-18T00:00:00Z,10020,52.0,0.1
2016-02-22T00:00:00Z,10115,55.0,0.18
2016-02-22T00:00:00Z,11371,52.0,4.02
2016-02-22T00:00:00Z,10017,52.0,0.12


In [0]:
%sql
--silver table
--view of weekly_stats table 
select * from weekly_stats limit 10;

week,avg_amount,avg_distance
2016-02-01T00:00:00Z,11.990339116719243,2.7463367507886396
2016-01-18T00:00:00Z,11.966793403573064,2.7421759047182723
2016-02-15T00:00:00Z,12.244146522870956,2.8944923763480825
2016-01-04T00:00:00Z,11.907765076862436,2.864603862830116
2016-02-08T00:00:00Z,12.20651356238698,2.751081374321874
2015-12-28T00:00:00Z,12.178038379530918,3.1040618336886974
2016-02-22T00:00:00Z,12.79211403184006,2.9734727878563483
2016-02-29T00:00:00Z,12.60960960960961,2.9733633633633634
2016-01-11T00:00:00Z,12.332039911308204,2.9312638580931316
2016-01-25T00:00:00Z,12.981361426256077,2.8746961102106963


In [0]:
%sql
--gold table
--view of top_n table
select * from top_n;

week,avg_amount,avg_distance,fare_amount,trip_distance,zip
2016-01-04T00:00:00Z,11.91,2.865,95.0,5.2,10009
2016-02-15T00:00:00Z,12.24,2.894,60.0,2.0,7311
2016-02-22T00:00:00Z,12.79,2.973,60.0,0.92,11422


### Validations

bronze layer validation

In [0]:
%sql
--verifying that there are no records with non-positive trip_distance.
select count(*) as invalid_rows from taxi_raw_records where trip_distance <= 0;

invalid_rows
0


silver layer validation


- flagged_rides table

In [0]:
%sql
--validating if there are only suspicious records in the flagged_rides table

select *
from flagged_rides f
Where not exists (
    select 1
    from taxi_raw_records r
    where f.week = date_trunc("week", r.tpep_pickup_datetime)
      and f.zip  = r.pickup_zip
      and (
           (r.pickup_zip = r.dropoff_zip and r.fare_amount > 50)
        or (r.trip_distance < 5 and r.fare_amount > 50)
      )
);


--it returns zero rows proving that there are no non-suspicious records in the flagged_rides table.


week,zip,fare_amount,trip_distance


In [0]:
%sql
--checking if there are any missing suspicious records 
--the records that are suspicious but not present in flagged_rides table.

select count(*) as missing_records from taxi_raw_records r
where (
    (r.pickup_zip = r.dropoff_zip AND r.fare_amount > 50) OR (r.trip_distance < 5 AND r.fare_amount > 50))
and not exists(
  select 1
  from flagged_rides f
  where f.week = date_trunc("week", r.tpep_pickup_datetime)
    and f.zip  = r.pickup_zip
  );

missing_records
0


- weekly_stats table

In [0]:
%sql
--checking if there any records with null values in the week column of the weekly_stats table
select *
from weekly_stats
WHERE week IS NULL;


week,avg_amount,avg_distance


In [0]:
%sql
--verifying values of silver table by recomputing them from bronze data
Select
  ws.week,
  ws.avg_amount,
  ws.avg_distance,
  AVG(r.fare_amount)    as recomputed_avg_amount,
  AVG(r.trip_distance)  as recomputed_avg_distance
From weekly_stats ws
JOIN taxi_raw_records r
  on ws.week = date_trunc('week', r.tpep_pickup_datetime)
Group by ws.week, ws.avg_amount, ws.avg_distance
having
     ABS(AVG(r.fare_amount)   - ws.avg_amount)    > 0.0001
  OR ABS(AVG(r.trip_distance) - ws.avg_distance)  > 0.0001;


week,avg_amount,avg_distance,recomputed_avg_amount,recomputed_avg_distance


gold layer validations

In [0]:
%sql
--gold table should only have top-3 highest fare rides
select * from top_n;
--the output should only have 3 records

week,avg_amount,avg_distance,fare_amount,trip_distance,zip
2016-01-04T00:00:00Z,11.91,2.865,95.0,5.2,10009
2016-02-15T00:00:00Z,12.24,2.894,60.0,2.0,7311
2016-02-22T00:00:00Z,12.79,2.973,60.0,0.92,11422


In [0]:
%sql
--counting the rows of top_n table, it should return 3.
select count(*) no_of_records from top_n ;

no_of_records
3
